In [109]:
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as py

import re

import tensorflow as tf
import numpy as np

import plotly.express as px

import matplotlib.pyplot as plt

# Data 불러오기

In [110]:
url = 'https://raw.githubusercontent.com/jihoo-kim/Data-Science-for-COVID-19/master/dataset/Patient/PatientInfo.csv'
Korea_patient = pd.read_csv(url, error_bad_lines=False)

url ='https://raw.githubusercontent.com/jihoo-kim/Data-Science-for-COVID-19/master/dataset/SearchTrend.csv'
Korea_Srch_Trend = pd.read_csv(url, error_bad_lines=False)

url = 'https://raw.githubusercontent.com/jihoo-kim/Data-Science-for-COVID-19/master/dataset/Time/Time.csv'
Korea_Time_Test = pd.read_csv(url, error_bad_lines=False)

url = 'https://raw.githubusercontent.com/jihoo-kim/Data-Science-for-COVID-19/master/dataset/Case.csv'
Korea_Case = pd.read_csv(url, error_bad_lines=False)

# Data 분석과 확인

In [111]:
Korea_patient.head()

,patient_id,global_num,sex,birth_year,age,country,province,city,disease,infection_case,infection_order,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,1000000001,2.0,male,1964.0,50s,Korea,Seoul,Gangseo-gu,NaN,overseas inflow,1.0,NaN,75.0,2020-01-22,2020-01-23,2020-02-05,NaN,released
1,1000000002,5.0,male,1987.0,30s,Korea,Seoul,Jungnang-gu,NaN,overseas inflow,1.0,NaN,31.0,NaN,2020-01-30,2020-03-02,NaN,released
2,1000000003,6.0,male,1964.0,50s,Korea,Seoul,Jongno-gu,NaN,contact with patient,2.0,2.002000e+09,17.0,NaN,2020-01-30,2020-02-19,NaN,released
3,1000000004,7.0,male,1991.0,20s,Korea,Seoul,Mapo-gu,NaN,overseas inflow,1.0,NaN,9.0,2020-01-26,2020-01-30,2020-02-15,NaN,released
4,1000000005,9.0,female,1992.0,20s,Korea,Seoul,Seongbuk-gu,NaN,contact with patient,2.0,1.000000e+09,2.0,NaN,2020-01-31,2020-02-24,NaN,released


In [112]:
Korea_Srch_Trend.head()

,date,cold,flu,pneumonia,coronavirus
0,2016-03-17,0.15554,0.34471,0.18181,0.01236
1,2016-03-18,0.14417,0.49416,0.17563,0.01027
2,2016-03-19,0.13290,0.39907,0.15145,0.01154
3,2016-03-20,0.13863,0.39662,0.14408,0.00945
4,2016-03-21,0.16735,0.56989,0.21199,0.00963


In [113]:
Korea_Time_Test.head()

,date,time,test,negative,confirmed,released,deceased
0,2020-01-20,16,1,0,1,0,0
1,2020-01-21,16,1,0,1,0,0
2,2020-01-22,16,4,3,1,0,0
3,2020-01-23,16,22,21,1,0,0
4,2020-01-24,16,27,25,2,0,0


In [114]:
Korea_Case.head()

,case_id,province,city,group,infection_case,confirmed,latitude,longitude
0,1000001,Seoul,Guro-gu,True,Guro-gu Call Center,79,37.508163,126.884387
1,1000002,Seoul,Dongdaemun-gu,True,Dongan Church,24,37.592888,127.056766
2,1000003,Seoul,Eunpyeong-gu,True,Eunpyeong St. Mary's Hospital,14,37.63369,126.9165
3,1000004,Seoul,Seongdong-gu,True,Seongdong-gu APT,13,37.55713,127.0403
4,1000005,Seoul,Jongno-gu,True,Jongno Community Center,10,37.57681,127.006


# Data 정제 및 1차 Linear Regression과 학습

In [115]:
tmp = Korea_Srch_Trend[1406:]

x1_data = np.array(tmp.coronavirus)
# x1_data = x1_data.reshape(1, -1)

In [116]:
tmp.tail()

,date,cold,flu,pneumonia,coronavirus
1456,2020-03-12,0.55771,0.10008,0.26335,20.56511
1457,2020-03-13,0.40744,0.07808,0.22799,19.29851
1458,2020-03-14,0.50816,0.06890,0.19108,16.62577
1459,2020-03-15,5.02730,0.06754,0.16644,14.92164
1460,2020-03-16,0.52925,0.12036,0.20826,16.89140


In [127]:
tmp = Korea_Time_Test

x_data = np.asarray(tmp.confirmed)
x_data = x_data.reshape(-1)

y_data =[]
y_data.append(0)
p = x_data[:len(tmp)-1]
y_data = np.hstack([y_data, p])

y_data = x_data - y_data

y_data = y_data.reshape(-1)

z_data = np.asarray(tmp.date)
z_data = z_data.reshape(-1)

# x_data
z_data

array(['2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23',
       '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27',
       '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31',
       '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04',
       '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08',
       '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12',
       '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16',
       '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20',
       '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24',
       '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28',
       '2020-02-29', '2020-03-01', '2020-03-02', '2020-03-03',
       '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-07',
       '2020-03-08', '2020-03-09', '2020-03-10', '2020-03-11',
       '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15',
       '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19',
       '2020-03-20'], dtype=object)

## 학습

#### 3B1B의 COVID-19 예상 식

In [121]:
learning_rate = 0.001
training_epochs = 1000
display_step = 100

num_of_samples = x_data.shape[0]

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W = tf.Variable(np.random.randn(), name="weight")
B = tf.Variable(np.random.randn(), name="bias")

pred = X + W * B * X

cost = tf.reduce_sum(tf.pow(pred-Y, 2))/(2*num_of_samples)

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    for (x, y) in zip(x_data, y_data):   # zip에 있는 리스트들을 원소로하는 튜플을 생성하여
        sess.run(optimizer, feed_dict={X: x, Y: y})

    if (epoch+1) % display_step == 0:
        c = sess.run(cost, feed_dict={X: x_data, Y: y_data})
        print("Epoch:", '%d' % (epoch+1), "cost=", "{:.1f}".format(c),
              "W=", sess.run(W), "b=", sess.run(B))

print("최적화가 완료되었습니다.")
training_cost = sess.run(cost, feed_dict={X: x_data, Y: y_data})
print("훈련이 끝난 후 비용과 모델 파라미터입니다.  cost=", training_cost,
      "W=", sess.run(W), "b=", sess.run(B), '\n')

fig = go.Figure()
fig.add_trace(go.Scatter(x=tmp.date, y=tmp.confirmed,
                    mode='markers', name='실제 Data'))
fig.add_trace(go.Scatter(x=tmp.date, y=(x_data + sess.run(W) * sess.run(B) * x_data)+x_data,
                    mode='lines',
                    name='예측 Data'))

Epoch: 100 cost= 18789.0 W= -0.6750831 b= 1.4280486
Epoch: 200 cost= 18795.3 W= -0.6783119 b= 1.421663
Epoch: 300 cost= 18795.3 W= -0.68166995 b= 1.4146553
Epoch: 400 cost= 18795.2 W= -0.6849926 b= 1.4077886
Epoch: 500 cost= 18795.1 W= -0.6882913 b= 1.4010375
Epoch: 600 cost= 18795.0 W= -0.6915718 b= 1.3943875
Epoch: 700 cost= 18795.0 W= -0.69484025 b= 1.3878239
Epoch: 800 cost= 18794.9 W= -0.69808763 b= 1.3813636
Epoch: 900 cost= 18794.8 W= -0.7013037 b= 1.3750253
Epoch: 1000 cost= 18794.8 W= -0.70448613 b= 1.36881
최적화가 완료되었습니다.
훈련이 끝난 후 비용과 모델 파라미터입니다.  cost= 18794.764 W= -0.70448613 b= 1.36881 



### 1차 선형회귀 모델 저장

In [123]:
saver = tf.train.Saver()

save_path = saver.save(sess, "./Save/First_Rinear.cpkt")

print('학습된 모델을 저장했습니다.')

학습된 모델을 저장했습니다.


### 1차 선형회귀 모델 복구

In [107]:
num_of_samples = x_data.shape[0]

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W = tf.Variable(np.random.randn(), name="weight")
B = tf.Variable(np.random.randn(), name="bias")

Given_Day = W*B*Given_Day

Future_Day = W*B*Given_Day

pred = X + W * B * X

cost = tf.reduce_sum(tf.pow(pred-Y, 2))/(2*num_of_samples)


optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# 1가지 변수를 입력 받습니다.
Test = int(input('검사 수: '))

# 세션에서 실행시켜야 훈련이 시작됩니다.
sess = tf.Session()
# 변수 W와 b가 앞에서 지정한 값으로 초기화됩니다.
sess.run(tf.global_variables_initializer())

# 저장된 학습 모델을 파일로부터 불러옵니다.
save_path = "./Save/First_Rinear.cpkt"

saver.restore(sess, save_path)


# 사용자의 입력 값을 이용해 배열을 만듭니다.
data = (Test)

arr = np.array(data, dtype=np.float32)



# 예측을 수행한 뒤에 그 결과를 출력합니다.
x_data = arr

dict = sess.run(hypothesis, feed_dict={X: x_data})

print(dict)

검사 수: 10000
INFO:tensorflow:Restoring parameters from ./Save/First_Rinear.cpkt


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_28' with dtype float and shape [?,1]
	 [[Node: Placeholder_28 = Placeholder[dtype=DT_FLOAT, shape=[?,1], _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: MatMul_4/_345 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_15_MatMul_4", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_28', defined at:
  File "c:\users\user\appdata\local\programs\python\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\user\appdata\local\programs\python\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.